# Random Forest

## Model Specification

Random Forest seeks to reduce the correlation among the base estimators and makes bagging more effective in reducing variance. Indeed, for $B$ $i.i.d.$ random variables each with variance $\sigma^2$, its average has variance $\frac{\sigma^2}{B}$. If the variables are simply $i.d.$ with positive pairwise correlation $\rho$, the variance of the average becomes $\rho\sigma^2+\frac{1-\rho}{B}\sigma^2$.

The training algorithm for random forest of trees works as follows:
1. For $b=1$ to $B$:
  
  (a) Draw a bootstrap sample $Z^{\ast}$ of size $N$ from the training data.
  
  (b) Grow a tree $T_b$ to the bootstrapped data, by recursively repeating the following steps for each terminal node of the tree, until the minimum node size $n_{min}$ is reached.
       
   - Select $m$ variables at random from the $p$ predictors
       
   - Pick the best variable/split-point among the $m$, in the usual way for trees
       
   - Split the node
       
2. Output the emsemble of trees $\{T_b\}^B_1$.

To do inference:

 - Regression: $\hat{f}_{rf}^B(x)=\frac{1}{B}\sum_{b=1}^BT_b(x)$.
 
 - Classification: Let $\hat{C}_b(x)$ be the class prediction of the $b$-th tree. Then $\hat{C}_{rf}(x)$= majority vote $\left\{\hat{C}_b(x)\right\}_{1}^B$.

### Variants and Generalizations

In `sklearn` a variant called **extremely randomized trees**, or **Extra Trees** is implemented. As in random forests, a random subset of candidate features is used, but instead of looking for the most discriminative thresholds, **thresholds are drawn at random for each candidate feature and the best of these randomly-generated thresholds is picked as the splitting rule**. This usually allows to reduce the variance of the model a bit more, at the expense of a slightly greater increase in bias - but it is hard to tell in advance.

## Theoretical Properties

### Advantages 

- Random forest can be fit in one sequence, with CV being performed along the way as Out-of-Bag (OOB) error estimate. The procedure is as simple as this. For each observation $z_i = (x_i, y_i)$, construct its random forest predictor by averaging only those trees corresponding to bootstrap samples in which $z_i$ did not appear. It can be shown that as the number of bootstrap sample $B$ gets large, the OOB error estimate for a random forest approaches its $N$-fold CV error estimate, and in the limit, the identity becomes exact (Don't have the exact detailed proof - but it should be a LLN type of argument).
- Most of the time overfitting does not happen, if there are enough trees in the forest (unverified claim).

### Disadvantages

- As improvement to bagging or vanilla decision trees, random forest also sacrafices on interpretability.
- The large number of trees can make the algorithm too slow and ineffective for real-time applications. In general, the algorithm is fast to train, but quite slow in making predictions.

### Relation to Other Models

- Random Forest is a generalization of bagging.

## Empirical Performance

### Advantages

- The empirical performance of random forest is often as good as boosting.
- Random forest requires less tuning as compared to boosting.
- The example in Section 15.2 in ESL finds that random forest uses relatively less trees than boosting to reach a reasonable prediction error.
- When the number of relevant variables increases, the performance of random forests is surprisingly robust to an increase in the number of noisy variables; see Section 15.3.4 in ESL. Intuitively, this robustness is due to the relative insensitivity of misclassification in each tree. In this sense, it is almost like random forest does its own feature seletion.

### Disadvantages

- Random forest or bagging in general does not work on linear estimates, while only nonlinear estimates such as trees benefit the most.
- The same example in Section 15.2 in ESL as mentioned above also shows that random forest is not capable of 'walking the last mile' even when we increase the number of trees.
- When the number of variables are large, but the fraction of relevant variables small, random forests are likely to perform poorly with small $m$, precisely because the relevant variables are not easy to be sampled. In fact, the empirical example in Section 15.3.4 in ESL indicates that as the probability gets small, the gap between boosting and random forest increases. Contrast this to one of the advantage point above.

## Implementation Details and Practical Tricks

**Random forest regressor in sklearn**

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

X, y = make_regression(n_features=4, n_informative=2,
                        random_state=0, shuffle=False)
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(X, y)
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
           max_features='auto', max_leaf_nodes=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)
print(regr.feature_importances_)
print(regr.predict([[0, 0, 0, 0]]))
print(regr.score([[0, 0, 0, 0]], [1]))

[ 0.17339552  0.81594114  0.          0.01066333]
[-2.50699856]
0.0


Many inputs and methods are the same as in [bagging](Bagging.ipynb) and [trees](CART.ipynb), except for the following

**`oob_score_`** : Score of the training dataset obtained using an out-of-bag estimate.

**Random forest classifiers in sklearn**

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=1000, n_features=4,
                            n_informative=2, n_redundant=0,
                            random_state=0, shuffle=False)
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X, y)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)
print(clf.feature_importances_)
print(clf.predict([[0, 0, 0, 0]]))
print(regr.score([[0, 0, 0, 0]], [1]))

[ 0.17287856  0.80608704  0.01884792  0.00218648]
[1]
0.0


**Choosing of $m$**

Typically values of $m$ are $\sqrt{p}$ or even as low as 1. For data with additive structure, empirically lower $m$ will benefit. The inventors recommend $\lfloor\sqrt{p}\rfloor$ and the minimum node size 1 for classification, and $\lfloor p/3\rfloor$ and minimum node size 5 for regression. In general, the lower the $m$, the better in de-correlating the trees, but it also induces more bias.

**Does random forest overfit as $B\rightarrow\infty$?**

Increasing $B$ has the effect of tending the random forest estimate to the limit:

$$\hat{f}_{rf}(x)=E_{\Theta}T(x; \Theta)=\lim_{B\rightarrow\infty}\hat{f}(x)^B_{rf}.$$

The distribution $\Theta$ here depends on the training data - in fact, since there is no pruning in growing the random forest, the averaged tree tends to be larger, resulting in a overly rich model and $E_{\Theta}T(x; \Theta)$ can overfit. But increasing $B$ does not help or prevent that. This is to constract with the number of trees in [boosting](boosting.ipynb), where the number of trees is a hyperparameter to tune to prevent overfitting.

**Extreme Randomized Trees in sklearn**

See the classes `ExtraTreesClassifier` and `ExtraTreesRegressor`. The interface/signature and methods are the same as their random forest cousins.

## Use Cases

## Results Interpretation, Metrics and Visualization

**Variable Importance**

**Variance importance plot** is constructed along the same line as in [boosting](boosting.ipynb), while in random forest the importance is accumulated over all the trees in the forest separately for each variable. The candidate split-variable selection increases the chance that any single variable gets included in a random forest, while it is possible that a single variable is never selected in the entire process of boosting.

Random forest is also capable of performing a different variable importance measure **for prediction**.

- When the $b$-th tree is constructed, the OOB sample is passed down the tree and the prediction accuracy is recorded.
- Then the value for the $j$-th variable is randomly permuted in the OOB samples, and accuracy is again recorded. Hence we have a record of decrease of prediction error, for variable $j$ for the $b$-th tree. Note that the $j$-th variable does not have to be chosen as the split variable in the $b$-th tree.
- The aforementioned error diff is averaged across all trees, and used as a measure of the importance of the $j$-th variable.

Intuitively, the randomization above effectively voids the effect of a variable, much like setting a coefficient to zero in a linear model. But note that this does not measure the effect on prediction were this variable not available.

A different version of variable importance measure is also implemented in `sklearn`. The **relative rank** (i.e. depth) of a feature used as a decision node in a tree can be used to assess the relative importance of that feature with respect to the predictability of the target variable. Features used at the top of the tree contribute to the final prediction decision of a larger fraction of the input samples. The expected fraction of the samples they contribute to can thus be used as an estimate of the relative importance of the features.

By averaging those expected activity rates over several randomized trees one can reduce the variance of such an estimate and use it for feature selection. Those estimates are stored as an attribute named `feature_importances_` on the fitted class, and it is shared by other tree-based classes in `sklearn`.

**Proximity Plot**

In growing a random forest, an $N\times N$ proximity matrix is accumulated for the training data. For every tree, any pair of OOB observations sharing a terminal node has their proximity increased by one. Then this proximity matrix is represented in two dimensions using multidimensional scaling.

Proximity plots for random forests often look very similar, irrespective of the data, which casts doubt in their utility (wondering if this is also because it uses MDS for low-dimension embedding). They are usually star shaped, one arm per class, and more prounounced the better the classificatoin performance. Also, it seems that points in the pure regions class-wise map to the extremities of the star, while points nearer the decision boundaries map nearer to the center.

## References

- ESL, Chapter 15
- [scikit-learn Document 1.11.2](http://scikit-learn.org/stable/modules/ensemble.html)
- < Hands-on Machine Learning >, Chapter 7

### Further Reading

- L. Breiman, “Random Forests”, Machine Learning, 45(1), 5-32, 2001.
- L. Breiman, “Arcing Classifiers”, Annals of Statistics 1998.
- P. Geurts, D. Ernst., and L. Wehenkel, “Extremely randomized trees”, Machine Learning, 63(1), 3-42, 2006.

## Misc.